# 📥 Récupération des Données - API OpenAgenda

Récupération, validation et export des événements depuis l'API OpenDataSoft.

**Étapes :**

1. Configuration et imports
2. Récupération des événements (API)
3. Validation des données (Pydantic)
4. Exploration rapide
5. Export (raw + processed)


## 1️⃣ Configuration


In [28]:
import os
import sys
import json
import time
from datetime import datetime, timedelta
from pathlib import Path
import logging

import requests
import pandas as pd
from tqdm.notebook import tqdm

# Ajouter le dossier racine au path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.config.settings import settings
from src.config.constants import (
    RAW_DATA_DIR,
    PROCESSED_DATA_DIR,
    RAW_EVENTS_FILE,
    PROCESSED_EVENTS_FILE,
)

# Configuration logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger(__name__)

print("✅ Configuration OK")

✅ Configuration OK


In [29]:
# Paramètres API
API_BASE_URL = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/evenements-publics-openagenda/records"

headers = {"Content-Type": "application/json"}

MAX_EVENTS = settings.max_events
LOCATION = "Marseille"  # Ville ciblée

print(f"🎯 Objectif : {MAX_EVENTS} événements à {LOCATION}")

🎯 Objectif : 10000 événements à Marseille


## 2️⃣ Récupération des Données


In [30]:
def fetch_openagenda_events(
    max_events: int = 100,
    location: str = None,
    timeout: int = 10,
    max_retries: int = 3,
) -> list[dict]:
    """
    Récupère les événements depuis l'API OpenDataSoft.

    Période couverte : événements terminés dans l'année passée + en cours + futurs.
    """

    all_events = []

    # Date de départ : il y a 1 an (inclut historique récent)
    date_past = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")

    params = {
        "limit": 100,
        "offset": 0,
        "order_by": "firstdate_begin asc",
    }

    # Construction du filtre WHERE
    where_clauses = []

    # Événements non terminés avant il y a 1 an
    where_clauses.append(f'lastdate_end >= "{date_past}"')

    # Filtre ville optionnel
    if location:
        where_clauses.append(f'location_city="{location}"')

    params["where"] = " AND ".join(where_clauses)

    print(f"📍 Filtre : {params['where']}")

    with tqdm(total=max_events, unit="evt", desc="📥 Récupération") as pbar:
        while len(all_events) < max_events:
            params["limit"] = min(100, max_events - len(all_events))

            for attempt in range(max_retries):
                try:
                    response = requests.get(
                        API_BASE_URL, params=params, headers=headers, timeout=timeout
                    )
                    response.raise_for_status()

                    data = response.json()
                    records = data.get("results", [])

                    # Log première requête
                    if params["offset"] == 0:
                        total = data.get("total_count", 0)
                        print(f"📊 Total disponible : {total}")

                    if not records:
                        logger.info(f"✅ {len(all_events)} événements récupérés")
                        return all_events

                    all_events.extend(records)
                    pbar.update(len(records))
                    params["offset"] += len(records)

                    if len(records) < params["limit"]:
                        logger.info(f"✅ {len(all_events)} événements récupérés")
                        return all_events

                    break

                except requests.exceptions.RequestException as e:
                    if attempt == max_retries - 1:
                        logger.error(f"❌ Échec après {max_retries} tentatives: {e}")
                        return all_events
                    time.sleep(2**attempt)

    return all_events

In [31]:
# Récupération
start_time = datetime.now()

raw_events = fetch_openagenda_events(
    max_events=MAX_EVENTS,
    location=LOCATION,
)

duration = (datetime.now() - start_time).total_seconds()

print(f"\n✅ {len(raw_events)} événements récupérés en {duration:.1f}s")
print(f"⚡ Vitesse : {len(raw_events) / duration:.1f} evts/sec")

📍 Filtre : lastdate_end >= "2025-01-16" AND location_city="Marseille"


📥 Récupération:   0%|          | 0/10000 [00:00<?, ?evt/s]

📊 Total disponible : 497


INFO: ✅ 497 événements récupérés



✅ 497 événements récupérés en 5.9s
⚡ Vitesse : 84.8 evts/sec


## 3️⃣ Validation Pydantic


In [32]:
from pydantic import BaseModel, Field, field_validator
from typing import Optional
from datetime import datetime


class Event(BaseModel):
    """Modèle Pydantic pour validation des événements."""

    uid: str
    title: str = Field(alias="title_fr")
    description: Optional[str] = Field(default=None, alias="description_fr")
    start_date: str = Field(alias="firstdate_begin")
    end_date: str = Field(alias="lastdate_end")  # ← UTILISER lastdate_end
    city: Optional[str] = Field(default=None, alias="location_city")
    address: Optional[str] = Field(default=None, alias="location_address")
    url: Optional[str] = Field(default=None, alias="canonicalurl")

    @field_validator("end_date")
    @classmethod
    def validate_end_date(cls, v):
        """Valide que l'événement n'est pas trop ancien."""
        date_limit = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")

        # Extraire seulement la date (YYYY-MM-DD) si ISO format complet
        event_date = v[:10] if len(v) > 10 else v

        if event_date < date_limit:
            raise ValueError(f"Événement trop ancien (terminé le {event_date})")

        return v

    class Config:
        populate_by_name = True

/var/folders/tv/qkptv90j2yj8jtfdqg5c16_00000gn/T/ipykernel_22782/1851160795.py:6: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class Event(BaseModel):


In [33]:
# Validation
validated_events = []
filtered_count = 0
errors = []

for event_data in tqdm(raw_events, desc="✅ Validation"):
    try:
        event = Event(**event_data)
        validated_events.append(event)
    except ValueError as e:
        # Événement filtré car trop ancien
        if "trop ancien" in str(e):
            filtered_count += 1
        else:
            errors.append({"uid": event_data.get("uid"), "error": str(e)})
    except Exception as e:
        errors.append({"uid": event_data.get("uid"), "error": str(e)})

print(f"\n✅ Validés : {len(validated_events)}")
print(f"🔍 Filtrés (trop anciens) : {filtered_count}")
print(f"❌ Erreurs : {len(errors)}")

✅ Validation:   0%|          | 0/497 [00:00<?, ?it/s]


✅ Validés : 497
🔍 Filtrés (trop anciens) : 0
❌ Erreurs : 0


## 4️⃣ Exploration


In [2]:
# Conversion en DataFrame
df = pd.DataFrame([event.model_dump() for event in validated_events])

print("📊 Aperçu des données :")
print(df[["title", "city", "start_date"]].head())

print(f"\n📏 Shape : {df.shape}")
print(f"\n📋 Colonnes : {list(df.columns)}")

NameError: name 'pd' is not defined

In [ ]:
# Statistiques rapides
print("📈 Distribution par ville :")
print(df["city"].value_counts().head(10))

print("\n📅 Période couverte :")
print(f"  Du {df['start_date'].min()} au {df['start_date'].max()}")

📈 Distribution par ville :
city
Marseille    497
Name: count, dtype: int64

📅 Période couverte :
  Du 2022-06-14T07:00:00+00:00 au 2032-01-01T01:00:00+00:00


## 5️⃣ Export


In [36]:
# Créer les dossiers si nécessaire
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Export données brutes (tel quel depuis l'API)
raw_path = RAW_DATA_DIR / RAW_EVENTS_FILE
with open(raw_path, "w", encoding="utf-8") as f:
    json.dump(raw_events, f, ensure_ascii=False, indent=2)

print(f"💾 Données brutes : {raw_path}")

# Export données traitées (validées Pydantic)
processed_path = PROCESSED_DATA_DIR / PROCESSED_EVENTS_FILE
processed_dict = [event.model_dump(mode="json") for event in validated_events]
with open(processed_path, "w", encoding="utf-8") as f:
    json.dump(processed_dict, f, ensure_ascii=False, indent=2)

print(f"💾 Données traitées : {processed_path}")

print("\n✅ Export terminé !")

💾 Données brutes : /Users/ppluton/Documents/Repositories/OC7---Projet-RAG-Assistant-Intelligent-Events/data/raw/events_raw.json
💾 Données traitées : /Users/ppluton/Documents/Repositories/OC7---Projet-RAG-Assistant-Intelligent-Events/data/processed/events_processed.json

✅ Export terminé !


## ✅ Résumé

- ✅ Récupération API OK
- ✅ Validation Pydantic OK
- ✅ Export raw + processed OK

**Prochaine étape** : Préparation des données pour le RAG (notebook 02)
